### Libraries

In [1]:
%%capture
%reset -f                        # clear all variables from the workspace
'generic imports'
import os
import pandas as pd
import sys
sys.path.append(os.path.abspath('..'))
from src import utils
import importlib
importlib.reload(utils)
from psutil import virtual_memory    
import datetime
import numpy as np                 

'machine learning imports'
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split


2023-11-03 10:40:15.054554: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-03 10:40:15.056000: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-03 10:40:15.080876: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-03 10:40:15.081488: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-03 10:40:15.534831: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
print('TensorFlow version:', tf.__version__)

TensorFlow version: 2.12.0


### GPU

In [3]:
# if gpu available print name, else use cpu
if tf.test.is_gpu_available():
    print('GPU:', tf.test.gpu_device_name())
else:
    print('CPU:', tf.config.list_physical_devices('CPU'))

CPU: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


2023-11-03 10:40:15.959751: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2023-11-03 10:40:15.959777: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: tiagociic
2023-11-03 10:40:15.959782: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: tiagociic
2023-11-03 10:40:15.959906: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.129.3
2023-11-03 10:40:15.959920: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.113.1
2023-11-03 10:40:15.959923: E tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:312] kernel version 535.113.1 does not match DSO version 535.129.3 -- cannot find working devices in this configuration


### Load Data

In [4]:
# Define the augmentation method and the data directory
AUGMENTATION = 'GReaT' # options: 'None', 'GReat', 'SMOTE', 'SMOTE-NC' or 'RealTabFormer'

data_dir = os.path.abspath('../data')

# Load the train and test datasets
df_train, df_test = utils.load_dataset(data_directory=data_dir, 
                                       augmentation=AUGMENTATION, 
                                       ignore_columns=['mbtcp.unit_id', 
                                                       'mbtcp.trans_id'])   

Loading complete.
Train data: 1500000 rows, 46 columns. 
Test data: 381934 rows, 46 columns.


### Data Preparation

In [5]:
# Creates X_train, y_train
X_train = df_train.drop(['Attack_label', 'Attack_type'], axis=1)
y_train = df_train['Attack_type']

# Creates X_test, y_test
X_test = df_test.drop(['Attack_label', 'Attack_type'], axis=1)
y_test = df_test['Attack_type']

#### Convert categorical features to one-hot encoded features

In [6]:
# Encode the training and test labels if needed
X_train_enc, X_test_enc, info = utils.encode_categorical(X_train, X_test)

Categorical features to be encoded:

http.request.method
mqtt.conack.flags
mqtt.topic
http.request.version
http.referer
dns.qry.name.len
mqtt.protoname

Encoding complete.
No of features before encoding: 44
No of features after encoding: 100


#### Label Encoding

In [7]:
y_train_enc, y_test_enc, le = utils.encode_labels(y_train, y_test)
y_train_bin = tf.keras.utils.to_categorical(y_train_enc)
# y_test_bin = tf.keras.utils.to_categorical(y_test_enc)

Attack_type and encoded labels:

Backdoor                0
DDoS_HTTP               1
DDoS_ICMP               2
DDoS_TCP                3
DDoS_UDP                4
Fingerprinting          5
MITM                    6
Normal                  7
Password                8
Port_Scanning           9
Ransomware              10
SQL_injection           11
Uploading               12
Vulnerability_scanner   13
XSS                     14


#### Standardization of Data

In [8]:
X_train_scaled, X_test_scaled = utils.scale_data(X_train_enc, X_test_enc, scaler_type='standard')

          mean      std       
Train:    -0.000    0.954     
Test:     -0.015    1.732     


### Model Training

In [9]:
# Define the model
model = Sequential()
model.add(Dense(256, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(len(le.classes_), activation='softmax')) 

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 

# ReduceLROnPlateau callback
monitor = tf.keras.callbacks.ReduceLROnPlateau(monitor="loss",
                                               factor=0.3,
                                               mode="min",
                                               patience=10,
                                               verbose=1,
                                               min_lr=1e-8)

# Checkpoint callback                                                
checkpoint = ModelCheckpoint(f'../checkpoints/neural_net/best_model_{AUGMENTATION}.h5', 
                              monitor='loss', 
                              save_best_only=True)

In [10]:
# Shuffle training data
X_train, y_train_bin = shuffle(X_train_scaled, y_train_bin, random_state=42)

# Train the model
history = model.fit(X_train, 
                    y_train_bin, 
                    epochs=100, 
                    batch_size=512, 
                    callbacks=[monitor, checkpoint])

### Model Evaluation

In [ ]:
# predict probabilities for test set and get the index of the highest probability
predictions = model.predict(X_test_scaled)

11936/11936 [==============================] - 9s 713us/step


In [ ]:
# get the index of the highest probability
predictions = tf.argmax(predictions, axis=1).numpy()

In [ ]:
# Calculate metrics 
accuracy = metrics.accuracy_score(y_test_enc, predictions)
precision_m = metrics.precision_score(y_test_enc, predictions, average='macro')
recall_m = metrics.recall_score(y_test_enc, predictions, average='macro')
f1_score_m = metrics.f1_score(y_test_enc, predictions, average='macro')
precision_w = metrics.precision_score(y_test_enc, predictions, average='weighted')
recall_w = metrics.recall_score(y_test_enc, predictions, average='weighted')
f1_score_w = metrics.f1_score(y_test_enc, predictions, average='weighted')

In [ ]:
# create dictionary for results
results = {
    "model": "Neural Net",
    "augmentations": AUGMENTATION,
    "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "accuracy": accuracy,
    "precision_macro": precision_m,
    "recall_macro": recall_m,
    "f1_macro": f1_score_m,
    "precision_weighted": precision_w,
    "recall_weighted": recall_w,
    "f1_weighted": f1_score_w
    }

# print results
utils.print_results_table(results)

╒══════════════════════╤═════════╕
│ Metric               │ Value   │
╞══════════════════════╪═════════╡
│ Accuracy             │ 93.59%  │
├──────────────────────┼─────────┤
│ Precision (macro)    │ 79.40%  │
├──────────────────────┼─────────┤
│ Recall (macro)       │ 78.80%  │
├──────────────────────┼─────────┤
│ F1 (macro)           │ 76.62%  │
├──────────────────────┼─────────┤
│ Precision (weighted) │ 94.90%  │
├──────────────────────┼─────────┤
│ Recall (weighted)    │ 93.59%  │
├──────────────────────┼─────────┤
│ F1 (weighted)        │ 93.70%  │
╘══════════════════════╧═════════╛


#### Save Metrics Results 

In [ ]:
# save results to csv   
utils.save_results_to_csv([results], '../results/metrics/neural_net.csv')

#### Confusion Matrix

In [ ]:
conf_mat = metrics.confusion_matrix(y_test_enc, predictions)

# Create a dataframe from the confusion matrix
conf_mat_df = pd.DataFrame(conf_mat, 
                            index = list(le.classes_), 
                            columns = list(le.classes_))
conf_mat_df.index.name = 'Actual'
conf_mat_df.columns.name = 'Predicted'


# Save the confusion matrix
conf_mat_df.to_csv(f"../results/conf_matrix/{results['model']}_{results['augmentations']}.csv")
conf_mat_df

Predicted,Backdoor,DDoS_HTTP,DDoS_ICMP,DDoS_TCP,DDoS_UDP,Fingerprinting,MITM,Normal,Password,Port_Scanning,Ransomware,SQL_injection,Uploading,Vulnerability_scanner,XSS
Actual,,,,,,,,,,,,,,,
Backdoor,4655,0,0,11,1,0,0,0,0,72,43,0,0,0,0
DDoS_HTTP,0,5923,0,0,0,0,0,0,0,0,0,0,0,10,3695
DDoS_ICMP,0,0,13452,0,1,48,0,0,0,0,0,0,0,0,0
DDoS_TCP,0,0,0,6501,0,0,0,0,0,3508,0,0,0,0,0
DDoS_UDP,0,0,0,0,24601,0,0,0,0,0,0,0,0,0,0
Fingerprinting,24,0,0,7,0,98,0,0,0,17,0,0,0,0,0
MITM,0,0,0,0,0,0,76,0,0,0,0,0,0,0,0
Normal,3,0,0,0,0,0,0,272761,6,0,0,0,5,0,1
Password,0,996,0,0,0,0,0,0,4897,0,0,4163,52,0,0
